# Prepare the dataset

In [ ]:
import urllib.request
import zipfile

In [ ]:
url = "https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip"

In [ ]:
file_name = "horse-or-human.zip"
training_dir = "horse-or-human/training/"

In [ ]:
urllib.request.urlretrieve(url,file_name)

('horse-or-human.zip', <http.client.HTTPMessage at 0x7f2e15e63d30>)

In [ ]:
zip_ref = zipfile.ZipFile(file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()


# Processing 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
      training_dir,
      target_size=(300, 300),
      class_mode='binary'
)

Found 1027 images belonging to 2 classes.


In [ ]:
validation_url = "https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip"
validation_file_name = "validation-horse-or-human.zip"
validation_dir = 'horse-or-human/validation/'
urllib.request.urlretrieve(validation_url, validation_file_name)
zip_ref = zipfile.ZipFile(validation_file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1/255)
validation_generator = train_datagen.flow_from_directory(
      validation_dir,
      target_size=(300, 300),
      class_mode='binary'
)


Found 256 images belonging to 2 classes.


In [ ]:
model = tensorflow.keras.models.Sequential([
      tensorflow.keras.layers.Conv2D(16, (3,3), activation='relu' ,
                  input_shape=(300, 300, 3)),
      tensorflow.keras.layers.MaxPooling2D(2, 2),
      tensorflow.keras.layers.Conv2D(32, (3,3), activation='relu'),
      tensorflow.keras.layers.MaxPooling2D(2,2),
      tensorflow.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tensorflow.keras.layers.MaxPooling2D(2,2),
      tensorflow.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tensorflow.keras.layers.MaxPooling2D(2,2),
      tensorflow.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tensorflow.keras.layers.MaxPooling2D(2,2),
      tensorflow.keras.layers.Flatten(),
      tensorflow.keras.layers.Dense(512, activation='relu'),
      tensorflow.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:

model.compile(loss='binary_crossentropy',
           optimizer=tensorflow.keras.optimizers.RMSprop(lr=0.001),
metrics=['accuracy'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
history = model.fit_generator(
      train_generator,
      epochs=15,
      validation_data=validation_generator,
)

<ipython-input-12-018d7e006abf>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/15
33/33 [==============================] - 77s 2s/step - loss: 0.6974 - accuracy: 0.6719 - val_loss: 1.7163 - val_accuracy: 0.7812
Epoch 2/15
33/33 [==============================] - 73s 2s/step - loss: 0.2696 - accuracy: 0.8890 - val_loss: 1.5867 - val_accuracy: 0.6719
Epoch 3/15
33/33 [==============================] - 72s 2s/step - loss: 0.1986 - accuracy: 0.9231 - val_loss: 1.4369 - val_accuracy: 0.7969
Epoch 4/15
33/33 [==============================] - 72s 2s/step - loss: 0.1020 - accuracy: 0.9688 - val_loss: 2.5961 - val_accuracy: 0.7539
Epoch 5/15
33/33 [==============================] - 72s 2s/step - loss: 0.0688 - accuracy: 0.9737 - val_loss: 2.7181 - val_accuracy: 0.7852
Epoch 6/15
33/33 [==============================] - 72s 2s/step - loss: 0.1623 - accuracy: 0.9620 - val_loss: 2.0004 - val_accuracy: 0.7461
Epoch 7/15
33/33 [==============================] - 72s 2s/step - loss: 0.0559 - accuracy: 0.9844 - val_loss: 2.0630 - val_accuracy: 0.7773
Epoch 8/15
33/33 [==

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 64)       0

### Testing Horse or Human

In [ ]:
import numpy
from google.colab import files
from keras.preprocessing import image 
from tensorflow.keras.utils import load_img , img_to_array

In [20]:
from keras.utils.image_dataset import image_utils
uploaded = files.upload()
for fn in uploaded.keys():
  # predicting images
  path='/content/'+fn
  img = load_img(path,target_size=(300,300))
  x = img_to_array(img)
  x = numpy.expand_dims(x,axis=0)
  image_tensor = numpy.vstack([x])
  classes = model.predict(image_tensor)
  print(classes)
  print(classes[0])
  if classes[0]>0.5:
    print(fn+ "is a human")
  else:
    print(fn+" is a horse")


Saving horse-2063672__340.jpg to horse-2063672__340.jpg
Saving woman-pic.webp to woman-pic.webp
1/1 [==============================] - 0s 40ms/step
[[0.]]
[0.]
horse-2063672__340.jpg is a horse
1/1 [==============================] - 0s 37ms/step
[[1.]]
[1.]
woman-pic.webpis a human
